In [ ]:
pip install evidently


In [35]:
import pandas as pd
train = pd.read_csv('/home/suhail/Downloads/ascarate-PM2.5_train.csv')
test = pd.read_csv('/home/suhail/Downloads/ascarate-PM2.5_test.csv')
train.columns = [col[1:-1] for col in train.columns]
test.columns = [col[1:-1] for col in test.columns]

#train = pd.read_csv('/home/suhail/Downloads/churn-bigml-80.csv')
#test = pd.read_csv('churn-bigml-20.csv')
print(train.columns)
train
test

Index(['NitricOxide', 'NitrogenDioxide', 'OxidesofNitrogen', 'Ozone',
       'WindSpeed', 'ResultantWindSpeed', 'ResultantWindDirection',
       'MaximumWindGust', 'Std.Dev.WindDirection', 'OutdoorTemperature',
       'DewPointTemperature', 'RelativeHumidity', 'SolarRadiation', 'PM2.5'],
      dtype='object')


,NitricOxide,NitrogenDioxide,OxidesofNitrogen,Ozone,WindSpeed,ResultantWindSpeed,ResultantWindDirection,MaximumWindGust,Std.Dev.WindDirection,OutdoorTemperature,DewPointTemperature,RelativeHumidity,SolarRadiation,PM2.5
0,2.5,7.7,9.0,51,13.6,13.4,283,22.0,10,60.1,14.2,16.3,0.000,19.5
1,1.6,10.8,12.0,48,15.1,14.8,290,28.0,13,58.8,15.8,18.2,0.008,22.2
2,1.0,13.2,14.5,47,20.1,19.8,284,41.4,9,58.6,16.5,18.9,0.145,31.3
3,2.5,4.9,6.0,41,4.7,4.4,124,10.9,19,65.4,40.0,39.2,0.000,11.0
4,2.6,3.9,5.1,43,5.8,5.6,124,13.3,17,64.6,43.3,45.8,0.000,11.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2.8,20.2,24.4,16,7.6,7.4,278,14.4,11,66.6,36.8,33.3,0.041,14.9
173,-1.5,3.8,3.9,33,8.4,8.1,295,16.5,15,68.0,37.2,32.5,0.262,11.1
174,-1.6,0.2,-0.1,37,9.6,9.2,289,18.2,18,68.9,36.5,30.4,0.580,11.4
175,-0.3,2.3,3.0,37,8.7,8.0,278,17.2,23,69.6,36.6,29.9,0.882,10.1


In [36]:
train['PM2.5'] = train['PM2.5'].apply(lambda x:1  if x > 35 else 0)
test['PM2.5'] = test['PM2.5'].apply(lambda x:1 if x > 35 else 0)

#train['Churn'] = train['Churn'].apply(lambda x: 1 if x == True else 0)
#test['Churn'] = test['Churn'].apply(lambda x: 1 if x == True else 0)

In [37]:
from evidently.dashboard import Dashboard
from evidently.tabs import  DataDriftTab
data_drift_report = Dashboard(tabs=[DataDriftTab()])
data_drift_report.calculate(train, test, column_mapping = None)
data_drift_report.save("/home/suhail/reports/my_report.html")

In [38]:
from sklearn.neighbors import *
X_train = train.drop('PM2.5', axis =1)
X_test = test.drop('PM2.5', axis =1)
y_train = train['PM2.5']
y_test = test['PM2.5']
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

KNeighborsClassifier()

In [39]:
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)
X_train['target'] = y_train
X_train['prediction'] = train_predictions
X_test['target'] = y_test
X_test['prediction'] = test_predictions

In [42]:
from evidently.pipeline.column_mapping import ColumnMapping
PM25_column_mapping = ColumnMapping()
PM25_column_mapping.target = 'target'
PM25_column_mapping.prediction = 'prediction'
PM25_column_mapping.numerical_features =train.drop('PM2.5', axis =1).columns

In [43]:
from evidently.tabs.base_tab import Verbose
from evidently.tabs import ClassificationPerformanceTab
PM25_model_performance_dashboard = Dashboard(tabs=[ClassificationPerformanceTab(verbose_level=Verbose.FULL)])
PM25_model_performance_dashboard.calculate(X_train, X_test, column_mapping = PM25_column_mapping)
PM25_model_performance_dashboard.save("/home/suhail/reports/classification_churn.html")